# EmOpti Workshop - Data Preparation for AutoML

Kernel `Python 3 (Data Science)` works well with this notebook

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
s3bucket = session.default_bucket()
s3prefix = "raa"
local_data_path = './data/emopti_data.csv'

role = get_execution_role()

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

## Data Preparation

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. 

If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. Autopilot is capable of handling datasets up to 5 GB.


Read the data into a Pandas data frame and take a look.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(local_data_path)
pd.set_option("display.max_columns", 500)  # Make sure we can see all of the columns
pd.set_option("display.max_rows", 10)  # Keep the output on one page
df.head(10)

In [ ]:
df.shape

There are some extraneous characters in the 'age' column, such as '<' and '>' and force the type to *int*

In [ ]:
df['age'] = df['age'].str.extract('(\d+)', expand=False)
df['age'] = df['age'].astype('int64')

In [ ]:
df.hist(figsize=(18,12))

In [ ]:
df['gender'].value_counts().plot(kind='bar')

In [ ]:
df['calc_disp'].value_counts().plot(kind='bar')

If we are going to use a Binary Classifier, then the label must contain only two values

In [ ]:
# drop all rows that are not ADMIT or DISCHARGE
df = df[df['calc_disp'] != 'OTHER']

The `admin_dest` column has a lot of missing values plus it has a very high correlation to our target label. Drop it.

Also, the *chief_complaint* field is text. For simplicity, drop it.

In [ ]:
df = df.drop(columns=['admit_dest', 'chief_complaint'])

Amazon SageMaker Autopilot takes care of preprocessing your data for you. You do not need to perform conventional data preprocssing techniques such as handling missing values, converting categorical features to numeric features, scaling data, and handling more complicated data types.

Moreover, splitting the dataset into training and validation splits is not necessary. Autopilot takes care of this for you. You may, however, want to split out a test set. That's next, although you use it for batch inference at the end instead of testing the model.


### Reserve some data for calling batch inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.


In [ ]:
df_train = df.sample(frac = 0.8, random_state=12345)

train_filename = 'train.csv'
df_train.to_csv(f'data/{train_filename}', index=False)

# save the text labels
test_labels = 'data/test_labels.csv'
df_test = df.drop(df_train.index)
df_test['calc_disp'].to_csv(test_labels, index=False, header=False)

# save the test data without the label column
df_test = df_test.drop(columns=['calc_disp'])
test_filename = 'test.csv'
df_test.to_csv(f'data/{test_filename}', index=False)
